In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
import pysam
import os

def Find_patient_information(patient_ID):
    patient_information = ["patient_ID","patient_type","IBD","day","month","year","seqID_virome"] 
    data_frame = pd.read_csv("PHAGE_BUBBLE/Freeze3_metadata.csv", header=0,
                             dtype=str, na_values=['NA'],
                             skip_blank_lines=True,
                             true_values=['true'], false_values=['false'],
                             nrows=None)

    
    patient_ID_whole_metadata = data_frame[data_frame["patient_ID"] == patient_ID] # Select all the row with the patient_ID
    selected_data = patient_ID_whole_metadata[patient_information] # select all the column within this list
    selected_data = selected_data.dropna(subset=["seqID_virome"]) # Remove all rows with 'NaN' in the SeqID column 
    return selected_data

def condense_data_file_and_time(data):
    Time_dict = {'Time': [f"{day}-{month}-{year}" for day, month, year in zip(data['day'], data['month'], data['year'])]}
    File_name_dict = {'File_name': [f"{seqID_virome}" for seqID_virome in data['seqID_virome']]}
    Time_dict.update(File_name_dict)
    return Time_dict
    
def write_output_for_Bash_command(data, patient_ID): # take data from the condense_data_file_and_time Function
    output_file_path = f'Time_series/Sample_output/{patient_ID}_sample_output.txt'
    with open(output_file_path, 'w') as file:
        for item in data["seqID_virome"]:
            if not pd.isna(item):
                file.write("%s\n" % item)
    
def Find_bamfiles(folder_path, FILE_ID):       
    file_list = [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if filename.startswith(tuple(FILE_ID)) and filename.endswith(".bam")
    ]
    return file_list

def Find_Library_Size(Library_size_matrix):
    data_frame = pd.read_csv(Library_size_matrix, sep='\t')
    Identification = data_frame.iloc[:,0]
                
    size_dict = {}
    for id_value in Identification:
        try:
            Library_Size = data_frame.loc[data_frame['Sample_name'] == id_value, 'Libary_Size'].values[0]
            size_dict[id_value] = Library_Size
        except IndexError:
            print(f"ID '{id_value}' not found in the table.")
            return None
    return size_dict

def Find_contigs(filepaths):
    contigs = []
    for record in SeqIO.parse(filepaths, "fasta"):
        contigs.append(record.id) 
    return contigs

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def Find_relative_abundance(component, Patient_ID, File_Time_dict):   
    File_name = File_Time_dict["File_name"]
    Dict_Sample_ID = Find_Library_Size("PHAGE_BUBBLE/sampleSeqCounts.tsv") # METAdata file, cannot change
    contigs = Find_contigs(f"PHAGE_BUBBLE/Paths/{component}_file/{component}_combined_file.fasta") # Component file location, fixed.
    File_name_lists = Find_bamfiles(f"/scratch/user/pham0323/Time_series/Patient_sample/{Patient_ID}/{component}", File_name) # location of bamfiles sorted, fixed.
    
    
    dict_data = {}
    for ID in File_name:
        bam_files = [file_path for file_path in File_name_lists if file_path.startswith(f"/scratch/user/pham0323/Time_series/Patient_sample/{Patient_ID}")]
        if ID not in dict_data:
            dict_data[ID] = {}
        for bam_file in bam_files:
            if ID in bam_file:
                # print(bam_file)
                with pysam.AlignmentFile(bam_file, 'rb') as bamfile:               
                    for references in bamfile.references:
                        mapped_reads = bamfile.count(reference=references,
                                                     read_callback=lambda read: not read.is_unmapped) 
                        genome_length = bamfile.get_reference_length(references)
                        for id_value, library_size in Dict_Sample_ID.items():
                            if id_value in bam_file:
                                Library_Size = library_size  
                            # print(f"{references}: mapped read = {mapped_reads}")
                            # print(f"Genome length = {genome_length}")
                        Rel_Ab = (mapped_reads * (1000000))/(Library_Size * genome_length)
                        # print(f"{ID}:{references}: Library_size = {Library_Size}: mapped read = {mapped_reads}: Rel_Ab = {Rel_Ab}")     
                        dict_data[ID][references] = Rel_Ab # Append the Relative abundance into the correct ID dictiondary in a list.
    return dict_data

def time_series(component, Patient_ID):
    Dict_1 = condense_data_file_and_time(Find_patient_information(Patient_ID)) # Dict_1 has patient's info skimmed down, it has Filename and Time.
    Dict_2 = Find_relative_abundance_scratch(component, Patient_ID, Dict_1)
    # print(Dict_2)
                # Dict_2 takes dict_1 info and create a dictionary with filename to paths to abundance.
    time_value = Dict_1["Time"]
                # create an array of time values

    # Create an array for each paths which contain abundance calculation
    contigs = Find_contigs(f"PHAGE_BUBBLE/Paths/{component}_file/{component}_combined_file.fasta")
    Paths_RAb = {}       
    # Create a date range
    date_range = pd.to_datetime(time_value, format='%d-%m-%Y')
    Paths_RAb['Date'] = date_range
    
    Mean_list = {}
    for contig in contigs: 
        value = [Dict_2[y][contig] for y in Dict_2.keys()]
        Paths_RAb[contig] = value    
        mean_number = np.mean(value)
        Mean_list[contig] = mean_number
        # print(f"The mean of {contig} = {mean_number}") 
    
    print(Paths_RAb)
    max_value = None
    max_categories = None
    
    for category, value in Mean_list.items():
        if max_value is None or value > max_value:
            max_value = value
            max_categories = category
    print(f"{max_categories} has the highest value at {max_value}")

    time_series_df = pd.DataFrame(Paths_RAb)
    # print(time_series_df)
    # Set the 'Date' column as the index for the time series
    time_series_df.set_index('Date', inplace=True)
    sns.set(style='whitegrid')
    plt.figure(figsize=(12,5))
    
    
    plt.rcParams['font.size'] = 18  
    plt.rcParams['axes.labelsize'] = 18
    plt.rcParams['axes.titlesize'] = 18
    plt.rcParams['xtick.labelsize'] = 13
    plt.rcParams['ytick.labelsize'] = 13
    for contig in contigs:
        ax = sns.lineplot(x='Date', y=contig, data=time_series_df, marker="o", label=contig)
        # Increase font size for x-axis label, and y-axis label
    ax.set_xlabel("Date")  # Adjust the fontsize as needed
    ax.set_ylabel("Relative Abundance")  # Adjust the fontsize as needed
    plt.title(f'Time Series of Abundance from {component} and of patient {Patient_ID}')
    plt.savefig(f"Time_series/image/{Patient_ID}/{component}_Time_series_Patient{Patient_ID}.png", dpi=300, bbox_inches='tight', format="png")
    plt.show()


In [5]:
import pandas as pd
import matplotlib.pyplot as plt

def Metadata_analysis(patient_ID):
    patient_information = ["patient_ID","IBD","day","month","year", "diagnosis", "FC_categories", "FC_categories_4", "inflammation", "flare", 
                           "type_flare", "disease_status", "current_medications", "seqID_virome"] 
    data_frame = pd.read_csv("PHAGE_BUBBLE/Freeze3_metadata.csv", header=0,
                             dtype=str, na_values=['NA'],
                             skip_blank_lines=True,
                             true_values=['true'], false_values=['false'],
                             nrows=None)

    patient_ID_whole_metadata = data_frame[data_frame["patient_ID"] == patient_ID]
    selected_data = patient_ID_whole_metadata[patient_information]
    selected_data = selected_data.dropna(subset=["seqID_virome"])

    
    selected_data['FC_categories'].replace(['highly_elevated','elevated','above_normal','normal','below_LOD'],[4,3,2,1,0],inplace=True)
    selected_data['FC_categories_4'].replace(['highly_elevated','elevated','above_normal','normal'],[4,3,2,1],inplace=True)
    selected_data['inflammation'].replace(['inactive','active'],[0,1],inplace=True)
    selected_data['flare'].replace(['no','yes'],[0,1],inplace=True)
    # print(selected_data)
    
    
    Dict_1 = { 'Time' : [f"{day}-{month}-{year}" for day, month, year in zip(selected_data['day'], selected_data['month'], selected_data['year'])]}
    # Dict_1['diagnosis'] = [diagnosis for diagnosis in selected_data['diagnosis']]
    Dict_1['FC_categories'] = [FC_categories for FC_categories in selected_data['FC_categories']]
    Dict_1['FC_categories_4'] = [FC_categories_4 for FC_categories_4 in selected_data['FC_categories_4']]
    Dict_1['inflammation'] = [inflammation for inflammation in selected_data['inflammation']]
    Dict_1['flare'] = [flare for flare in selected_data['flare']]
    
    from datetime import datetime
    Dict_1['Time'] = [datetime.strptime(date, '%d-%m-%Y') for date in Dict_1['Time']]

    sorted_data_tuples = sorted(zip(*Dict_1.values()), key=lambda x: x[0])
    
    Dict_1 = {'Time': [item[0] for item in sorted_data_tuples], 'FC_categories': [item[1] for item in sorted_data_tuples], 
              'FC_categories_4': [item[2] for item in sorted_data_tuples], 'inflammation': [item[3] for item in sorted_data_tuples], 
              'flare': [item[4] for item in sorted_data_tuples]}
    
    return Dict_1

def column_graph(patient):
    data = Metadata_analysis(patient)
    data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y')
    print(data)
    df = pd.DataFrame(data)
    # df = df.sort_values(by = 'Time')
    df.set_index('Time', inplace=True)
    
    df.plot(kind='bar', figsize=(30, 6), grid=True)
    plt.xlabel('Time Points')
    plt.ylabel('Values')
    plt.title(f'Time Series Bar Graph for Different Categories of {patient}')
    plt.legend(title='Categories')
    # plt.savefig(f"{patient}_Col_Graph.png", dpi=300, bbox_inches='tight', format="png")
    # Rotate x-axis labels for better readability (optional)
    plt.xticks(rotation=0)

    # Show the graph
    plt.show()
    
    

In [6]:
All_patient_ID = ['230_4','282','1012A','2093A','2028A','2037A', '2064A','2065A','228','203','2036A','2032A','2056A','2057A','2066A','234','268']
with open("resolved_component_name.txt", "r") as file: 
    line = file.read().splitlines()
Component = line
# Results of the above: 
# Component = ['phage_comp_5', 'phage_comp_10', 'phage_comp_21', 'phage_comp_24', 'phage_comp_26', 'phage_comp_27', 'phage_comp_28',
#              'phage_comp_31', 'phage_comp_45', 'phage_comp_46', 'phage_comp_65', 'phage_comp_73', 'phage_comp_76', 'phage_comp_86',
#              'phage_comp_87', 'phage_comp_93', 'phage_comp_105', 'phage_comp_122', 'phage_comp_142']

def Patient_Time_series(patient):
    for component in Component: 
        print(component)
        time_series(component, patient)
        
def Patient_Time_series_scratch(patient):
    for component in Component: 
        print(component)
        time_series_scratch(component, patient)
    
def Component_Time_series(component):
    for patient in patient_ID_list:
        print(patient)
        time_series(component,patient)
    for patient_scratch in patient_ID_list_scratch:
        print(patient_scratch)
        time_series_scratch(component, patient_scratch)
